# Problems of Chapter 3

## 3.2

In [ ]:
import tabulate

from .. import des


def normalize_table(table):
    # DES table -> customary Python dictionary
    return {
        input_pos: output_pos for output_pos, input_pos in enumerate(table, start=1)
    }


IP = normalize_table(des.INITIAL_PERMUTATION)
FP = normalize_table(des.FINAL_PERMUTATION)

tabulate.tabulate(
    [[pos, IP[pos], FP[IP[pos]]] for pos in range(1, 6)],
    headers=["pos", "IP(pos)", "FP(IP(pos))"],
    tablefmt="html",
)

## 3.4

In [ ]:
plaintext = key = 0xFFFF_FFFF_FFFF_FFFF

data = des.permute(plaintext, 64, des.INITIAL_PERMUTATION)
k1 = next(des.key_schedule(key))
L0, R0 = des.split_at_right(data, 32)
L1, R1 = R0, L0 ^ des.f(R0, k1)

print(f"(L1, R1) = ({L1:08X}, {R1:08X})")

## 3.6

In [ ]:
import math
from itertools import accumulate


PC1 = normalize_table(des.PERMUTED_CHOICE1)
PC2 = normalize_table(des.PERMUTED_CHOICE2)


def rotate_pos_left(i: int, rot: int) -> int:
    return (i - rot - 1) % 28 + 1


def which_s_box_at_pos(i: int) -> int:
    return math.ceil(i / 6)


pos_after_LS = list(accumulate(des.ROTATIONS, rotate_pos_left, initial=PC1[1]))
pos_after_LS.pop(0)

pos_after_PC2 = [PC2.get(i) for i in pos_after_LS]

affected_S_boxes = [
    which_s_box_at_pos(i) if i is not None else i for i in pos_after_PC2
]

tabulate.tabulate(
    zip(range(1, 17), pos_after_LS, pos_after_PC2, affected_S_boxes),
    headers=["round", "pos after LS", "pos after PC-2", "affected S-box"],
    missingval="N/A",
    tablefmt="html",
)

## 3.10

We can express the clock frequency required to achieve a given data rate as:
$$\text{clock frequency} = \frac{\text{data rate}}{\text{bits per cycle}}$$

The bits per cycle ratio for the given setup is:
$$\frac{1 \text{ iteration}} {1 \text{ clock cycle}} \cdot \frac{64 \text{ bits}} {16 \text{ iterations}} = 4 \text{ bit/cycle}$$

Clock frequency required for encrypting a fast network link running at a speed of 1 Gb/sec:
$$\frac{1 \text{ Gbit/sec}}{4 \text{ bit/cycle}} = 250\text{ MHz}$$

Clock frequency required to support a speed of 8 Gb/sec:
$$\frac{8 \text{ Gbit/sec}}{4 \text{ bit/cycle}} = 2\text{ GHz}$$

## 3.12

In [ ]:
from datetime import timedelta
from string import ascii_uppercase

import humanize


def average_key_search_time(key_space: int, key_test_speed: int = 10**6) -> str:
    return humanize.naturaldelta(timedelta(seconds=key_space / key_test_speed) / 2)

In [ ]:
# 8 randomly chosen 8-bit ASCII characters
print(average_key_search_time(2**56))  # 64 bits - 8 LSB

In [ ]:
# 8 randomly chosen 7-bit ASCII characters
print(average_key_search_time(2**48))  # 64 bits - 8 LSB - 8 MSB

In [ ]:
# only uppercase letters w/o LSB
key_space = {chr(ord(letter) >> 1) for letter in ascii_uppercase}
print(f"{key_space = }")
print(average_key_search_time(len(key_space) ** 8))